In [1]:
from qiskit import QuantumCircuit, transpile
import numpy as np

num_qubits = 8
qc = QuantumCircuit(num_qubits)
num_layers = int(np.ceil(np.log2(num_qubits)))


In [2]:
from qcc.ml import USE_CUDA

USE_CUDA


False

In [3]:
from itertools import zip_longest, tee

def convolve(wires):
    qc.rz(0.1, wires[0])
    qc.sx(wires[0])
    qc.rz(0.1, wires[0])
    qc.sx(wires[0])
    qc.rz(0.1, wires[0])
    
    qc.rz(0.1, wires[1])
    qc.sx(wires[1])
    qc.rz(0.1, wires[1])
    qc.sx(wires[1])
    qc.rz(0.1, wires[1])
    
    qc.cnot(wires[0], wires[1])
    qc.ry(0.1, wires[0])
    qc.rz(0.1, wires[1])
    qc.cnot(wires[1], wires[0])
    qc.ry(0.1, wires[0])
    qc.cnot(wires[0], wires[1])
    
    qc.rz(0.1, wires[0])
    qc.sx(wires[0])
    qc.rz(0.1, wires[0])
    qc.sx(wires[0])
    qc.rz(0.1, wires[0])
    
    qc.rz(0.1, wires[1])
    qc.sx(wires[1])
    qc.rz(0.1, wires[1])
    qc.sx(wires[1])
    qc.rz(0.1, wires[1])
        
def pool(wires):
    qc.crz(0.1, wires[0], wires[1])
    qc.x(wires[0])
    qc.crx(0.1, wires[0], wires[1])

def _convolution(iterable):
    a, b = tee(iterable)
    first = next(b, None)
    lst = list(zip_longest(a, b, fillvalue=first))
    last = lst.pop()[::-1]
    lst = lst[::2] + lst[1::2]

    if len(lst) > 1:
        lst.insert(0, last)

    for wires in lst:
        convolve(wires)

def _pooling(iterable):
    measurements = iterable[1::2]
    controlled = iterable[0::2]

    for wires in zip(measurements, controlled):
        pool(wires)

    return controlled

wires = qc.qubits
for _ in range(num_layers):
    _convolution(wires)
    wires = _pooling(wires)


In [4]:
qc.draw('mpl')


KeyboardInterrupt: 

In [ ]:
qc = transpile(qc, basis_gates=["cx", "sx", "x", "ry", "rx", "rz"], optimization_level=0)
qc.depth(), sum(qc.count_ops().values())


(135, 415)

In [ ]:
135, 415
188, 593


(188, 593)

In [ ]:
qc = QuantumCircuit(num_qubits)

def _ry_pool(iterable):
    measurements = iterable[1::2]
    controlled = iterable[0::2]

    for wires in zip(measurements, controlled):
        qc.cry(0.1, wires[0], wires[1])

    return controlled

wires = qc.qubits
for _ in range(num_layers):
    wires = _ry_pool(wires)
    
qc = transpile(qc, basis_gates=["cx", "sx", "x", "ry", "rx", "rz"], optimization_level=0)
qc.depth(), sum(qc.count_ops().values())


(12, 28)

In [6]:
from pathlib import Path

path = Path("/Users/mingyoungjeng/Documents/Spring 2023.nosync/quantum-convolutional-classifier/test/tmp")
for file in path.glob("**/mqcc*/*.*"):
    stem = file.stem.replace("mqcc_", "mqcc-optimized_")
    # items = file.stem.split("_")
    # items.insert(2, "16x16")
    # stem = "_".join(items)
    
    # items = file.parent.stem.split("_")
    # items.insert(2, "16x16")
    # parent = "_".join(items)
    
    new_path = file.parent.with_stem(file.parent.stem.replace("mqcc", "mqcc_optimized"))
    new_path.mkdir(parents=True, exist_ok=True)
    
    new_path = new_path / f"{stem}{file.suffix}"
    file.rename(new_path)


In [13]:
from pathlib import Path

path = Path("/Users/mingyoungjeng/Downloads/results_thesis/_raw_results/CIFAR10")
dest = path / "32x32"

for file in path.glob("**/*_32x32_*.*"):
    parent = dest / file.parent.stem
    parent.mkdir(parents=True, exist_ok=True)
    
    new_path = parent / file.name
    file.rename(new_path)


In [20]:
import numpy as np

test = np.array([[[0, 1], [2, 3]], [[4, 5], [6, 7]]])
print(test)

test.reshape(4, 2)


[[[0 1]
  [2 3]]

 [[4 5]
  [6 7]]]


array([[0, 1],
       [2, 3],
       [4, 5],
       [6, 7]])

In [5]:
from pathlib import Path
from PIL import Image

filename = Path("../img/color/Jayhawk_512x512x3.jpg")

with Image.open(filename, "r") as im:
    im.crop((0  , 0  , 256, 256)).save("tl.png")
    im.crop((128, 0  , 384, 256)).save("tc.png")
    im.crop((256, 0  , 512, 256)).save("tr.png")
    
    im.crop((0  , 128, 256, 384)).save("cl.png")
    im.crop((128, 128, 384, 384)).save("cc.png")
    im.crop((256, 128, 512, 384)).save("cr.png")
    
    im.crop((0  , 256, 256, 512)).save("bl.png")
    im.crop((128, 256, 384, 512)).save("bc.png")
    im.crop((256, 256, 511, 512)).save("br.png")
